In [20]:
import os

curr_dir = os.listdir(".")

extensions = (".pdf", ".docx", ".pptx")

curr_dir_files = [file for file in curr_dir if file.endswith(extensions)]
print(curr_dir_files)

MODEL = "llama3.2"

['Cover Letter.docx', 'EP-1.pdf', 'EP-2.pdf', 'samplepptx.pptx']


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import UnstructuredPowerPointLoader


pages = []

for PDF_FILE in curr_dir_files:
    loader = PyPDFLoader(PDF_FILE)
    pages += loader.load()

loader2 = Docx2txtLoader("Cover Letter.docx")
pages2 = loader2.load()
print(pages2)

loader3 = UnstructuredPowerPointLoader("samplepptx.pptx")
pages3 = loader3.load()
print(pages3)



[Document(metadata={'source': 'Cover Letter.docx'}, page_content='Dear Hiring Manager,\n\nI am excited to apply for the Associate Full Stack Developer position at ABBYY. With hands-on experience in full-stack development and a background at Iron Mountain, a competitor in the industry, I bring a unique perspective and a proven ability to contribute to innovative solutions.\n\nAt Iron Mountain, I developed and optimized applications using technologies such as ReactJS, NodeJS, ExpressJS, and MongoDB. A notable project was DevTinder, a full-stack application where I created RESTful APIs, implemented JWT-based authentication, and ensured database integrity through robust data validation. Leveraging tools like Redux for state management and Tailwind CSS for responsive designs, I focused on delivering efficient and user-centric solutions. Additionally, my experience with real-time problem-solving in cloud-based environments has sharpened my ability to tackle challenges effectively.\n\nI admir

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)
chunks = splitter.split_documents(pages)



In [4]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

# Pull the model first

# Create the embedding and vectorstore
embedding = OllamaEmbeddings(model="llama3.2:3b")
vectorstore = FAISS.from_documents(chunks, embedding)

C:\Users\sanka\AppData\Local\Temp\ipykernel_21984\1393954115.py:7: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings(model="llama3.2:3b")


In [5]:
retriever = vectorstore.as_retriever()
retriever.invoke("What is framework?")

[Document(id='f778f4a7-57f6-4b85-9099-3a0583b98fe2', metadata={'source': 'EP-1.pdf', 'page': 1}, page_content='Inception! \ue081 Namast e-R e act )\n2\nSo Let’s begin our Namaste React Journey  \nIn this course we will study how the React \nconcepts are actually applied into the industry \ni.e. into the real world projects.\nSo are you ready to fall in love with React????\nIntroducing React.\nQ ) What is React? Why React is known as ‘React’?\nReact is a JavaScript Library. The name ‘React’ was chosen \nbecause the library was designed to allow developers to react \nto changes in state and data within an application, and to \nupdate the user interface in a declarative and efficient \nmanner.\nQ ) What is Library?\nLibrary is a collections of prewritten code snippets that can \nbe used and reused to perform certain tasks. A particular \nJavaScript library code can be plugged into application code \nwhich leads to faster development and fewer vulnerabilities to \nhave errors.\nExamples: R

In [6]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="llama3.2:3b", temperature=0)
model.invoke("Who is Prime minister of India?")

AIMessage(content="I'm not aware of the current Prime Minister of India as my knowledge cutoff is December 2023. However, I can suggest some ways for you to find out the current Prime Minister of India:\n\n1. Check online news websites: You can check online news websites such as BBC News, Al Jazeera, or The New York Times for the latest updates on the Indian government.\n2. Visit the official website of the Government of India: You can visit the official website of the Government of India (www.gov.in) to get information about the current Prime Minister and other government officials.\n3. Check social media: You can also check the official social media accounts of the Prime Minister's Office or the Government of India to get updates on the current Prime Minister.\n\nAs of my knowledge cutoff in December 2023, the Prime Minister of India was Narendra Modi. However, please note that this information may have changed since then.", additional_kwargs={}, response_metadata={'model': 'llama3.2

In [7]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser

print(chain.invoke("Who is Prime minister of India?"))

I'm not aware of the current Prime Minister of India as my knowledge cutoff is December 2023. However, I can suggest some ways for you to find out the current Prime Minister of India:

1. Check online news websites: You can check reputable news websites such as BBC News, Al Jazeera, or The Hindu for the latest updates on the Indian government.
2. Visit the official website of the Government of India: You can visit the official website of the Government of India (www.gov.in) to get information about the current Prime Minister and other government officials.
3. Check social media: You can also check the official social media accounts of the Prime Minister's Office or the Government of India to get updates on the current Prime Minister.

As of my knowledge cutoff in December 2023, the Prime Minister of India was Narendra Modi. However, please note that this information may have changed since then.


In [8]:
from langchain.prompts import PromptTemplate

template ="""
You are an assistant that provides answers to questions based on
a given context. 

Answer the question based on the context. If you can't answer the
question, reply "I don't know".

Be as concise as possible and go straight to the point.

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Reactjs is a JavaScript library for building user interfaces", question="What is Reactjs?"))




You are an assistant that provides answers to questions based on
a given context. 

Answer the question based on the context. If you can't answer the
question, reply "I don't know".

Be as concise as possible and go straight to the point.

Context: Reactjs is a JavaScript library for building user interfaces

Question: What is Reactjs?



In [9]:
chain = prompt | model | parser

chain.invoke(
    {"context": "Reactjs is a JavaScript library for building user interfaces", "question": "What is Reactjs?"}
)

'Reactjs is a JavaScript library for building reusable UI components.'

In [10]:
from operator import itemgetter

chain = (
    {
    "context" : itemgetter("question") | retriever,
    "question" : itemgetter("question"),
    }
    | prompt
    | model
    | parser
    
)

In [11]:
question = input("Enter your question: ")

print(f"Question: {question}")
print(f"Ansewer : {chain.invoke({'question': question})}")
print("************************")

Question: What is this about? summarise the content here
Ansewer : This appears to be a document containing notes and questions from an EP (Electron Packager) tutorial, likely related to building web applications with React. The content includes boilerplate code for HTML and CSS, discussion on creating a "Hello World" program using only HTML or JavaScript, and explanations of various optimization techniques such as image optimization, caching, compression, and tree shaking.
************************
